In [16]:
import keras.utils
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
import pandas as pd
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.model_selection import train_test_split
from keras.utils import pad_sequences
import numpy as np
from sklearn.metrics import classification_report
from collections import Counter
import joblib


In [45]:
data = pd.read_csv('combined_data.csv')

# Split the data into features (X) and labels (y)
X = data['Nhận xét đánh giá'].values
y = data['Cảm xúc'].values
label_encoder = LabelEncoder()

# Encode the labels
y_encoded = label_encoder.fit_transform(y)
joblib.dump(y_encoded, "label_encoder.pkl")

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=132)



In [6]:
borderline_smote = BorderlineSMOTE(sampling_strategy='auto', random_state=132)

max_words = 5000
max_len = 1000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

X_train_seq = pad_sequences(X_train_seq, maxlen=max_len)
X_val_seq = pad_sequences(X_val_seq, maxlen=max_len)
X_train_resampled, y_train_resampled = borderline_smote.fit_resample(X_train_seq, y_train)

In [6]:
class_distribution_before_SMOTE = Counter(y_train)

class_distribution_after_SMOTE = Counter(y_train_resampled)

print("Class Distribution Before SMOTE:")
for label, count in class_distribution_before_SMOTE.items():
    print(f"Class {label}: {count} samples")

# Print class distribution after SMOTE
print("\nClass Distribution After SMOTE:")
for label, count in class_distribution_after_SMOTE.items():
    print(f"Class {label}: {count} samples")

Class Distribution Before SMOTE:
Class 4: 2992 samples
Class 3: 2607 samples
Class 5: 3587 samples
Class 0: 3939 samples
Class 1: 912 samples
Class 2: 1251 samples

Class Distribution After SMOTE:
Class 4: 3939 samples
Class 3: 3939 samples
Class 5: 3939 samples
Class 0: 3939 samples
Class 1: 3939 samples
Class 2: 3939 samples


In [13]:

# Build and compile your CNN model
embedding_dim = 120
num_filters = 128
kernel_size = 5
num_classes = len(np.unique(y_train_resampled))
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(Conv1D(num_filters, kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(num_classes, activation='softmax'))

optimizer = keras.optimizers.Adam(learning_rate=0.0007)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [15]:
batch_size = 128
epochs = 100

model.fit(X_train_seq, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val_seq, y_val))

Epoch 1/100
120/120 [==============================] - 114s 950ms/step - loss: 0.4645 - accuracy: 0.8393 - val_loss: 0.8316 - val_accuracy: 0.7039
Epoch 2/100
120/120 [==============================] - 113s 946ms/step - loss: 0.4379 - accuracy: 0.8488 - val_loss: 0.8432 - val_accuracy: 0.7042
Epoch 3/100
120/120 [==============================] - 110s 915ms/step - loss: 0.4158 - accuracy: 0.8553 - val_loss: 0.8534 - val_accuracy: 0.7083
Epoch 4/100
120/120 [==============================] - 114s 952ms/step - loss: 0.3935 - accuracy: 0.8682 - val_loss: 0.8729 - val_accuracy: 0.7104
Epoch 5/100
120/120 [==============================] - 123s 1s/step - loss: 0.3762 - accuracy: 0.8751 - val_loss: 0.8923 - val_accuracy: 0.7039
Epoch 6/100
120/120 [==============================] - 120s 998ms/step - loss: 0.3571 - accuracy: 0.8813 - val_loss: 0.9134 - val_accuracy: 0.7031
Epoch 7/100
120/120 [==============================] - 126s 1s/step - loss: 0.3448 - accuracy: 0.8842 - val_loss: 0.9334 

In [16]:
y_pred = model.predict(X_val_seq)
# Convert numerical labels to original labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_labels)

# Convert true labels to original labels
y_val_labels = label_encoder.inverse_transform(y_val)

# Generate classification report
report = classification_report(y_val_labels, y_pred_labels)
print(report)

120/120 [==============================] - 6s 44ms/step
                 precision    recall  f1-score   support

    bình thường       0.63      0.49      0.55       977
không liên quan       0.93      0.96      0.94       230
         rất tệ       0.70      0.75      0.73       325
        rất tốt       0.53      0.56      0.55       632
             tệ       0.87      0.81      0.84       789
            tốt       0.58      0.71      0.64       870

       accuracy                           0.67      3823
      macro avg       0.71      0.71      0.71      3823
   weighted avg       0.67      0.67      0.67      3823



In [17]:
model.save("Sentiment_Analysis_Model.h5")

In [46]:
custom_text = "tối cảm thấy sản phẩm rất tệ"

# Tokenize and pad the custom text
custom_text_seq = tokenizer.texts_to_sequences([custom_text])
custom_text_seq = keras.utils.pad_sequences(custom_text_seq, maxlen=max_len)

# Make predictions on the custom text
custom_text_pred = model.predict(custom_text_seq)

# Convert numerical labels to original labels
custom_text_pred_label = np.argmax(custom_text_pred, axis=1)
custom_text_pred_label = label_encoder.inverse_transform(custom_text_pred_label)

# Print the predicted label
print(f"Predicted Sentiment: {custom_text_pred_label[0]}")

1/1 [==============================] - 0s 60ms/step
Predicted Sentiment: rất tốt


1/1 [==============================] - 0s 59ms/step
Predicted Sentiment: bình thường
